Task 1 Part 1

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime
import os
import re
import openpyxl
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

In [2]:
driver_path='C:\\Users\\Lenovo\\OneDrive\\Documents\\chromedriver.exe'

In [3]:
options = webdriver.ChromeOptions()
#All are optional
options.add_experimental_option("detach", True)
options.add_argument("--disable-extensions")
options.add_argument("--disable-notifications")
options.add_argument("--disable-Advertisement")
options.add_argument("--disable-popup-blocking")
options.add_argument("start-maximized")


In [4]:
service = Service(driver_path)
driver = webdriver.Chrome(options=options,service=service)

In [6]:
url_path="https://www.youtube.com/@UnfoldDataScience/videos"
driver.get(url_path)
time.sleep(5)

In [8]:
item = []
SCROLL_PAUSE = 1
last_height = driver.execute_script("return document.documentElement.scrollHeight")

count = 170

while count > len(item):
    driver.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")
    time.sleep(SCROLL_PAUSE)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")

    if new_height == last_height:
        break
    last_height = new_height


data = []
try:
    for e in WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div#details'))):
    
        vurl = e.find_element(By.CSS_SELECTOR,'a#video-title-link').get_attribute('href')
        data.append(vurl)
except:
    pass


In [9]:
len(data)

411

In [10]:
comments=[]
likes_count=[]
title=[]
views=[]
upload_date=[]
for i in range(0,10):
    driver.get(data[i])
    time.sleep(7)
    likes_count.append(driver.find_element(By.XPATH,'.//*[@id="segmented-like-button"]/ytd-toggle-button-renderer/yt-button-shape/button/div[2]').text)
    title.append(driver.find_element(By.CLASS_NAME,'style-scope ytd-watch-metadata').text)
    more_button = driver.find_element(By.CSS_SELECTOR,'#expand')
    more_button.click()
    views.append(driver.find_element(By.XPATH,'.//*[@id="info"]/span[1]').text)
    upload_date.append(driver.find_element(By.XPATH,'.//*[@id="info"]/span[3]').text)
    driver.execute_script("return scrollBy(0,6000);")
    time.sleep(7)
    comments.append(WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,"//h2[@id='count']/yt-formatted-string"))).text)



In [11]:
def extract_title(data):
  new_data = []
  for string in data:
    new_data.append(string[:string.find("|")].strip())
  return new_data



In [12]:
titles=extract_title(title)


In [13]:


def extract_int_value(comments):
  int_values = []
  for comment in comments:
    comment = re.sub(r"[^\d\-+\.]", "", comment)
    int_value = float(comment)
    int_values.append(int_value)

  return int_values

In [14]:
comments_=extract_int_value(comments)
views_=extract_int_value(views)
likes_=extract_int_value(likes_count)

In [15]:
data={'Title':titles,
      'Views':views_,
      'Likes':likes_,
      'Comments':comments_,
      'Date':upload_date}

In [16]:
dataframe=pd.DataFrame(data)
dataframe

,Title,Views,Likes,Comments,Date
0,Simplifying Confidence Intervals,1083.0,41.0,11.0,"Sep 14, 2023"
1,Cost of Living in London,1623.0,56.0,15.0,"Sep 6, 2023"
2,6 Activation functions for deep learning,1262.0,46.0,10.0,"Sep 3, 2023"
3,5 python packages to wow your client,1593.0,75.0,11.0,"Aug 24, 2023"
4,Junior data scientist to Senior Data Scientist,1330.0,61.0,16.0,"Aug 21, 2023"
5,Encode decoder seq 2 seq architecture,1452.0,54.0,5.0,"Aug 17, 2023"
6,Participate and learn more,760.0,32.0,14.0,"Aug 10, 2023"
7,How to Pass AWS ML specialty exam,3570.0,137.0,6.0,"Aug 3, 2023"
8,LSTM next word prediction in Python,2314.0,88.0,15.0,"Jul 24, 2023"
9,LSTM explained simply,4704.0,195.0,38.0,"Jul 19, 2023"


In [17]:
dataframe.to_csv('extracted.csv')

In [19]:
dg=pd.read_csv('extracted.csv')

dg

,Unnamed: 0,Title,Views,Likes,Comments,Date
0,0,Simplifying Confidence Intervals,1083.0,41.0,11.0,"Sep 14, 2023"
1,1,Cost of Living in London,1623.0,56.0,15.0,"Sep 6, 2023"
2,2,6 Activation functions for deep learning,1262.0,46.0,10.0,"Sep 3, 2023"
3,3,5 python packages to wow your client,1593.0,75.0,11.0,"Aug 24, 2023"
4,4,Junior data scientist to Senior Data Scientist,1330.0,61.0,16.0,"Aug 21, 2023"
5,5,Encode decoder seq 2 seq architecture,1452.0,54.0,5.0,"Aug 17, 2023"
6,6,Participate and learn more,760.0,32.0,14.0,"Aug 10, 2023"
7,7,How to Pass AWS ML specialty exam,3570.0,137.0,6.0,"Aug 3, 2023"
8,8,LSTM next word prediction in Python,2314.0,88.0,15.0,"Jul 24, 2023"
9,9,LSTM explained simply,4704.0,195.0,38.0,"Jul 19, 2023"


In [20]:
dg['Date'] = pd.to_datetime(dg['Date'])



In [21]:
import pandas as pd


def avg_views(d1, d2):

  avg_views_df = dg.loc[(dg['Date'] > '2023-08-15')]

  total_views = avg_views_df['Views'].sum()

  num_videos = avg_views_df.shape[0]
  avg_views = total_views / num_videos

  return avg_views

print(avg_views(dg['Views'], dg['Date']))


1390.5


In [22]:
#Identify the video with the highest likes-to-views ratio.
def highest_ratio(data):  
    for i in range(1,len(data)):
         if data['Likes'][i]/data['Views'][i]>data['Likes'][i-1]/data['Views'][i-1]:
            return data['Title'][i]
print(highest_ratio(dg))

6 Activation functions for deep learning


In [23]:
#Find the correlation between the number of likes and the number of comments for the videos.
corr=dg['Likes'].corr(dg['Comments'])
print(corr)


0.6562831137481018


In [24]:
# Determine the most common day of the week for video uploads.
dg['Day'] = dg['Date'].dt.day_name()
dg['Day'].value_counts().idxmax()
    

'Thursday'

In [25]:
#Detect any outliers in the views count.
dg['z_score'] = (dg['Views'] - dg['Views'].mean())/dg['Views'].std()
dg.loc[dg['z_score'] > 3]

,Unnamed: 0,Title,Views,Likes,Comments,Date,Day,z_score


Task 1 Part 2


In [36]:
!pip install requests
!pip install beautifulsoup4


In [39]:
url_path=["https://www.imdb.com/search/title/?title_type=feature&year=2013,2023&sort=num_votes,desc",
     "https://www.imdb.com/search/title/?title_type=feature&year=2013-01-01,2023-12-31&sort=num_votes,desc&start=51&ref_=adv_nxt",
     "https://www.imdb.com/search/title/?title_type=feature&year=2013-01-01,2023-12-31&sort=num_votes,desc&start=101&ref_=adv_nxt",
     "https://www.imdb.com/search/title/?title_type=feature&year=2013-01-01,2023-12-31&sort=num_votes,desc&start=151&ref_=adv_nxt"]

In [40]:
import requests
from bs4 import BeautifulSoup
import csv
from collections import defaultdict

In [41]:
movies=[]
ratings=[]
for i in url_path:
    response=requests.get(i)
    soup=BeautifulSoup(response.content,'html.parser')
    result=soup.find(class_="lister-list")
    movie_element=result.find_all(class_="lister-item mode-advanced")
    for m_e in movie_element:
        title=m_e.find(class_="lister-item-header").find('a').get_text()
        year=int(m_e.find("span",class_="lister-item-year text-muted unbold").get_text().replace('(','').replace(')','').replace('I','').replace(' ','').replace('V','').replace('X',''))
        director=m_e.find("p",class_="").find('a').get_text()
        genre=m_e.find("span",class_="genre").get_text().replace('\n','').replace(' ','')
      
      
        movies.append({
            "Title": title,
            "Release_year":year,
            "Director": director,
            "Genre": genre
        })
    rating=soup.find_all("div",class_="inline-block ratings-imdb-rating")
    for r in rating:
        ratings.append(float(r.find('strong').get_text()))   

In [42]:
df=pd.DataFrame(movies)

In [43]:
df['Rating']=ratings

In [44]:
df.to_csv('movies.csv')

In [45]:
#1
avg_rating=df[df['Rating']>8.0]['Rating'].mean()
print(avg_rating)

8.260714285714284


In [46]:
#The most common genre among the top-rated movies.
common_genre=df[df['Rating']>8.0]['Genre'].value_counts().idxmax()
print(common_genre)

Biography,Drama,History


In [47]:
#Identify the director with the highest average IMDb rating.
director_rating=df.groupby('Director')['Rating'].mean().sort_values(ascending=False).head(1)
print(director_rating)

Director
Vivek Agnihotri    8.6
Name: Rating, dtype: float64


In [48]:
#Determine the year with the highest number of top-rated movies.
year_rating=df[df['Rating']>8.0]['Release_year'].value_counts().idxmax()
print(year_rating)

2019


Task 2

In [27]:
import pandas as pd

In [26]:
data = pd.read_csv('C:\\Users\\Lenovo\\OneDrive\\Documents\\house.csv')
print("First few rows of the dataset:")
print(data.head())

First few rows of the dataset:
      crim    zn  indus  chas    nox     rm   age     dis  rad  tax  ptratio  \
0  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296     15.3   
1  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242     17.8   
2  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242     17.8   
3  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222     18.7   
4  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222     18.7   

        b  lstat  medv  
0  396.90   4.98  24.0  
1  396.90   9.14  21.6  
2  392.83   4.03  34.7  
3  394.63   2.94  33.4  
4  396.90   5.33  36.2  


In [34]:
# Discretize the "age" variable into three bins
age_bins = [0, 25, 50, float('inf')]
age_labels = ['Young', 'Middle-aged', 'Old']
data['age_group'] = pd.cut(data['age'], bins=age_bins, labels=age_labels)

In [33]:
data['is_charles_river'] = data['chas'].apply(lambda x: 'Yes' if x == 1 else 'No')

In [32]:
# Define a function to remove outliers using IQR
def remove_outliers(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return column[(column >= lower_bound) & (column <= upper_bound)]

# Remove outliers for each numerical column in the dataset
numerical_columns = ['crim', 'zn', 'indus', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat', 'medv']
for col in numerical_columns:
    data[col] = remove_outliers(data[col])

In [31]:

window_size = 3
data['rm_smoothed'] = data['rm'].rolling(window=window_size, min_periods=1).mean()

data['tax_normalized'] = (data['tax'] - data['tax'].min()) / (data['tax'].max() - data['tax'].min())
data['lstat_normalized'] = (data['lstat'] - data['lstat'].min()) / (data['lstat'].max() - data['lstat'].min())

In [28]:
# Perform a simple linear regression to predict the median value of "medv" based on "rm"
from sklearn.linear_model import LinearRegression

X = data[['rm']]
y = data['medv']

# Create and fit the regression model
model = LinearRegression()
model.fit(X, y)

# Print the coefficients and intercept of the regression model
print("\nLinear Regression Model Coefficients:")
print(f"Coefficient (slope): {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")




Linear Regression Model Coefficients:
Coefficient (slope): 9.102108981180308
Intercept: -34.670620776438554


In [29]:
from sklearn.impute import SimpleImputer

# Replace missing values with the mean of the column
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)


In [30]:
data.dropna(subset=['rm'], inplace=True)  # Removes rows with missing 'rm' values
X = data[['rm']]
y = data['medv']
